In [ ]:
# Documentação
# https://python-twitter.readthedocs.io/en/latest/twitter.html#module-twitter.api

## Importando bibliotecas

In [ ]:
!pip install python-twitter

In [1]:
import twitter
import json

In [2]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

## Configurando api

In [ ]:
# Variaveis de acesso, keys cedidas pela api do twitter
# tokens.json

In [54]:
# Carregando o arquivo de configuração
with open('tokens.json') as f:
    data = json.load(f)

In [55]:
# Carregando a API do Twitter, utilizando os dados do arquivo json
api = twitter.Api(**data)

In [56]:
# Testar se tudo está certo

# Caso tiver alguma credencial errada, o resultado será:
#twitter.error.TwitterError: [{'code': 32, 'message': 'Could not authenticate you.'}]

print(api.VerifyCredentials())

{"created_at": "Tue Jul 26 22:26:37 +0000 2016", "default_profile": true, "default_profile_image": true, "friends_count": 1, "id": 758066017742512128, "id_str": "758066017742512128", "name": "Malu Freitas", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "MaluMalufrt"}


In [57]:
# A API do Twitter permite fazer diversos tipos de consultas. O retorno dessas consultas pode ser acessado facilmente. 

# Função que, a partir de uma lista de status, mostra o nome do usuário e o texto do twitter.

def print_status(status_list):
    for status in status_list:
        print('(' + str(status.created_at) + ') ' + str(status.user.name) + ':' + str(status.text) + '\n')

## Buscando dados do twitter

In [58]:
# Busca por termos

status_list = api.GetSearch(term="mi band 4",
                            lang='pt',
                            count=100)  #,
                            #result_type='mixed')

## Criando lista com os tweets

In [59]:
def cria_tweet_list(status_list):
    tweet_list = []
    
    for tweet in status_list:
        tweet_list.append(tweet.text)
    
    return tweet_list

In [60]:
# Printando elementos
print_status(status_list=status_list)

(Thu Oct 17 00:23:34 +0000 2019) ANTI-HERÓI:Acho q minha mi band 4 chega amanhã

(Wed Oct 16 23:17:29 +0000 2019) Leo Silva:Xiaomi Mi Band 4 Smart Bracelet ( China Version ) - Black
🏬 Loja: #Gearbest 
🔖 Cupom: GBXMSH01
💵 Preço: R$104,99
💳… https://t.co/Fvfz4SeoFA

(Wed Oct 16 22:41:11 +0000 2019) Di Orc 🧟‍♂️:@canalzigueira @amazonBR A mão coça pra pegar o Mi Band 4

(Wed Oct 16 22:38:56 +0000 2019) satan himself:RT @CapitaoTurbo: Alguém ai tem uma MI BAND 4  que possa me ajudar por favor?

(Wed Oct 16 22:36:46 +0000 2019) Renata Klein nova pobre:Pensando em pegar a mi band 4
Sou muito xiaomir

(Wed Oct 16 22:24:48 +0000 2019) Tɑbɑtɑ:Black Friday abaixe o preço do mi band 4 te imploro

(Wed Oct 16 22:09:05 +0000 2019) Leo Silva:Xiaomi Mi Router 4 Dual Band 2.4G 5G 1167Mbps Gigabit Wireless WiFi Router  Sale - Banggood Mobile
🏬 Loja:… https://t.co/SQpBgPcxrn

(Wed Oct 16 22:08:57 +0000 2019) Leo Silva:Xiaomi Mi Band 4 Smart Bracelet International Version - Black
🏬 Loja: #Gearbest 
🔖 Cupo

In [61]:
print(len(status_list))

100


In [62]:
tweet_list = cria_tweet_list(status_list)

In [63]:
print(tweet_list)

['Acho q minha mi band 4 chega amanhã', 'Xiaomi Mi Band 4 Smart Bracelet ( China Version ) - Black\n🏬 Loja: #Gearbest \n🔖 Cupom: GBXMSH01\n💵 Preço: R$104,99\n💳… https://t.co/Fvfz4SeoFA', '@canalzigueira @amazonBR A mão coça pra pegar o Mi Band 4', 'RT @CapitaoTurbo: Alguém ai tem uma MI BAND 4  que possa me ajudar por favor?', 'Pensando em pegar a mi band 4\nSou muito xiaomir', 'Black Friday abaixe o preço do mi band 4 te imploro', 'Xiaomi Mi Router 4 Dual Band 2.4G 5G 1167Mbps Gigabit Wireless WiFi Router  Sale - Banggood Mobile\n🏬 Loja:… https://t.co/SQpBgPcxrn', 'Xiaomi Mi Band 4 Smart Bracelet International Version - Black\n🏬 Loja: #Gearbest \n🔖 Cupom: GBBAND4MI\n💵 Preço: R$118,… https://t.co/JXrkBZPT11', 'Alguém ai tem uma MI BAND 4  que possa me ajudar por favor?', 'nossa eu preciso de uma MI Band 4\n\né a pulseira da responsabilidade', 'Comprei minha mi band 4 :)', 'To tao triste\nAlguem me da a mi band 4 pfvr', 'Mi Band 4 280$$ na loja de acessórios se tá maluco mais fácil colo

In [ ]:
for i in tweet_list:
    print("'" + str(i) + "'")
    print('----------------------------------------------------------------------------------------------------------------')

In [ ]:
# criei
# tweet_list
# tweet_list_2
# tweet_list_3

## Remove os tweets duplicados

In [64]:
lista_sem_duplicados = list(set(tweet_list))

In [65]:
print(type(lista_sem_duplicados))

<class 'list'>


In [66]:
print(len(tweet_list))

100


In [67]:
print(len(lista_sem_duplicados))

95


## Coloca os tweets da lista em um arquivo .csv

In [68]:
def cria_arquivo_csv(lista):
    arquivo = open('tweets_mi_band_4.csv', 'w', encoding='utf-8')
    
    for pos_tweet in range(len(lista)):
        arquivo.write('"' + str(lista[pos_tweet]) + '"')
        arquivo.write('\n')
        
    arquivo.close()

In [69]:
cria_arquivo_csv(lista_sem_duplicados)

## Cria um data frame

In [43]:
# Criando data frame
df = pd.read_csv('classificado_tweets_mi_band_4.csv', encoding='utf-8')

In [20]:
#df = pd.read_csv('Tweets_Mg.csv', encoding='utf-8')

In [44]:
df.head()

,tweets,classes
0,Tive que desativar o negócio do sono da mi ban...,negativo
1,"RT @cachorro_frio: @fernandobentto cara, jura?...",negativo
2,"@FelipeDuarte151 Tenho a mi band 3, mas acho a...",positivo
3,To vendendo minha pulseira mi band 4(relógio)😉,neutro
4,mi band 4 na internet = 160 reais \nmi band 3 ...,neutro


In [46]:
tweets = df['tweets']
classes = df['classes']

## Limpeza dos dados

In [24]:
# Para limpeza dos dados
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    stopwords.remove('não')
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

In [ ]:
Preprocessing('Eu não gosto do partido, e também não votaria novamente nesse governante! http://python.w3.pt/?p=234')

## Tokenização

In [26]:
from nltk.tokenize import TweetTokenizer

In [27]:
tweet_tokenizer = TweetTokenizer()

In [33]:
frase = 'A live do @blogminerando é show! :) :-) ;) =D'

In [34]:
tweet_tokenizer.tokenize(frase)

['A',
 'live',
 'do',
 '@blogminerando',
 'é',
 'show',
 '!',
 ':)',
 ':-)',
 ';)',
 '=D']

## Vetorização - Bag of words

In [47]:
# Instancia o objeto que faz a vectorização
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [48]:
# Aplica o vetorizador nos dados
freq_tweets = vectorizer.fit_transform(tweets)
type(freq_tweets)

scipy.sparse.csr.csr_matrix

In [49]:
# Formato linha, coluna da matriz
freq_tweets.shape

(178, 943)

In [50]:
# Treino do modelo de machine learning
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [51]:
freq_tweets.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Testando o modelo

In [52]:
print(tweets)

0      Tive que desativar o negócio do sono da mi ban...
1      RT @cachorro_frio: @fernandobentto cara, jura?...
2      @FelipeDuarte151 Tenho a mi band 3, mas acho a...
3         To vendendo minha pulseira mi band 4(relógio)😉
4      mi band 4 na internet = 160 reais \nmi band 3 ...
                             ...                        
173    descobri que eu só tive 44 minutos de sono pro...
174    mi band 4 é pior que o zika virus jesus a cada...
175    RT @CapitaoTurbo: Alguém ai tem uma MI BAND 4 ...
176                          comprei um mi band 4 é isso
177     @heyanab Meu mi Band 4 tá no limbo faz um tempão
Name: tweets, Length: 178, dtype: object


In [72]:
print(testes)

0      Tive que desativar o negócio do sono da mi ban...
1      RT @cachorro_frio: @fernandobentto cara, jura?...
2      @FelipeDuarte151 Tenho a mi band 3, mas acho a...
3         To vendendo minha pulseira mi band 4(relógio)😉
4      mi band 4 na internet = 160 reais \nmi band 3 ...
                             ...                        
137              meu mi Band 4 chegou e eu tô toda besta
138    esse negocio de mi band 4 me calou porque é le...
139    @gustavokuhl Sei lá eu só tenho a mi band 4 e ...
140    @canalrssv mi band 4 realmente e uma escolha b...
141    Ela chegou! Xiaomi Mi Band 4 desembarca no Bra...
Name: tweets, Length: 142, dtype: object


In [200]:
# Testando o modelo
testes = tweets[:20]
print(tweets[1])

RT @cachorro_frio: @fernandobentto cara, jura? com o preço que tu comprou dava pra ter pego um xiaomi mi 9, um mi band 4, uma caixa jbl ult…


In [201]:
# Aplicando a função de pre processamento nos testes
testes = [Preprocessing(i) for i in testes]

In [202]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(testes)

In [203]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

desativ negóci son mi band 4 bagulh tav deix doid já,, positivo
rt @cachorro_fri @fernandobentt cara, jura? preç compr dav pra ter peg xiaom mi 9, mi band 4, caix jbl ult…, neutro
@felipeduarte151 mi band 3, ach 4 legal cont tel color, positivo
to vend puls mi band 4(relógio😉, neutro
mi band 4 internet = 160 real mi band 3 jipar = 260 real kkkkkpi, neutro
ganh mi band 4 😍 ah deu, positivo
vontad compr mi band 4 tá gigantesca, sort não cart crédit não far beste, positivo
@xiaomibrasil par tent tal mi band 4 ?!?!, neutro
quer sab mi band 4 boa resist quant arranh tal, pq n ach lig nenhum sobr isso…, negativo
algu xiaom mi band 4 pres, positivo
próx compr vai ser mi band 4, positivo
@buzsheep fic bom mi band 4 😍, positivo
mi band 4 qualqu cois mundo&gt&gt&gt&gt&gt&gt&gt, positivo
tô port xiaomi, mi band 4 mão 🤪, positivo
certo, admito, vici produt xaom vou compr mi band 4, xiaom mi 8 lit redm airdots…, positivo
@mizanzik viu sit vári tem diferentes?, neutro
compr não compr mi band 4, dúvi

In [205]:
# Probabilidades de cada classe
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

['negativo' 'neutro' 'positivo']


array([[0.05, 0.43, 0.52],
       [0.3 , 0.39, 0.3 ],
       [0.01, 0.21, 0.78],
       [0.  , 0.73, 0.27],
       [0.  , 0.97, 0.03],
       [0.01, 0.05, 0.94],
       [0.11, 0.03, 0.86],
       [0.  , 0.88, 0.12],
       [0.95, 0.02, 0.02],
       [0.03, 0.32, 0.65],
       [0.01, 0.1 , 0.89],
       [0.  , 0.04, 0.96],
       [0.03, 0.46, 0.5 ],
       [0.  , 0.17, 0.83],
       [0.  , 0.13, 0.87],
       [0.07, 0.91, 0.02],
       [0.05, 0.28, 0.67],
       [0.01, 0.79, 0.19],
       [0.03, 0.32, 0.65],
       [0.01, 0.03, 0.96]])

## Criando modelos com Pipelines

In [83]:
# Pipelines são interessantes para reduzir código e automatizar fluxos
from sklearn.pipeline import Pipeline

In [206]:
pipeline_svm_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', svm.SVC(kernel='linear'))
])

## Validando os Modelos com Validação Cruzada

In [207]:
# Fazendo o cross validation do modelo
resultados = cross_val_predict(pipeline_svm_simples, tweets, classes, cv=10)

In [208]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(classes,resultados)

0.6460674157303371

In [196]:
# Medidas de validação do modelo
sentimento=['positivo','negativo','neutro']
print (metrics.classification_report(classes,resultados,sentimento))

              precision    recall  f1-score   support

    positivo       0.64      0.72      0.68        79
    negativo       0.62      0.48      0.54        27
      neutro       0.66      0.62      0.64        72

    accuracy                           0.65       178
   macro avg       0.64      0.61      0.62       178
weighted avg       0.65      0.65      0.64       178



In [197]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   negativo  neutro  positivo  All
Real                                     
negativo        13       5         9   27
neutro           4      45        23   72
positivo         4      18        57   79
All             21      68        89  178


In [198]:
def Metricas(modelo, tweets, classes):
  resultados = cross_val_predict(modelo, tweets, classes, cv=10)
  return 'Acurácia do modelo: {}'.format(metrics.accuracy_score(classes,resultados))

In [199]:
# svm linear simples
Metricas(pipeline_svm_simples,tweets,classes)

'Acurácia do modelo: 0.6460674157303371'

In [134]:
# svm linear simples
Metricas(pipeline_simples,tweets,classes)

'Acurácia do modelo: 0.6573033707865169'